In [12]:
from json import load, dump
import os
import re
import numpy as np
import pandas as pd

In [15]:
results = []

for file in os.listdir("short_paper"):
    try:
        with open(f"short_paper/{file}", "r", encoding="utf-8") as f:
            model = load(f)
    except:
        print(file)
        continue
    cnt_plus = 0
    cnt_neg = 0
    print(len(model))
    for instance in model:
        option = re.findall(r'\d+', instance["raw_model_output"])
        option = [int(x) for x in option if int(x) in range(5)]
        # instance["predicted_label"] = option[-1] if option else -1
        if len(option) > 1:
            cnt_plus += 1
        elif len(option) == 0:
            cnt_neg += 1
    # with open(f"new_method/{file}.json", "w", encoding="utf-8") as f:
    #     dump(model, f)
    results.append({"model": file, "more than one number": cnt_plus, "no valid numbers": cnt_neg})

df = pd.DataFrame(results)

qwen3_14b_semeval
12379
12379
12379
2210
2210
2210
2210
2210
12379
12379
2210
2210
12379
2210
2210
12379
2210
12379
2210
2210
2210
2210
2210
12379
2210
2210
2210
2210
12379
12379
12379
2210
2210
2210
12379
12379
2210
12379
12379
12379
12379
2210
12379
2210
2210
12379
12379
2210
12379
12379
2210
12379
2210
12379
12379
12379
2210
12379


In [14]:
df

,model,more than one number,no valid numbers
0,qwen3_8b_semeval,12338,2
1,gpt-oss_20b_semeval,2,8
2,llama3.2_latest_semeval,0,114
3,gemma2_2b_sst,0,0
4,llama3.1_8b_sst,2,1
5,gemma2_9b_sst,0,0
6,qwen2.5_14b_sst,0,0
7,qwen2.5_7b_sst,0,0
8,aya_8b_semeval,0,0
9,phi3.5_3.8b_semeval,3620,5222
